In [ ]:
!pip install koreanize-matplotlib
!pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install mecab-python3

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.nouns("복잡하지만 설치 완료"))

['설치', '완료']


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import koreanize_matplotlib
from konlpy.tag import Mecab

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/haram4th/data4mdai/main/hotelscom_review_train.csv")
# test_data = pd.read_csv("../06machine_learning/data/hotelscom_review_test.csv")

In [ ]:
train_data.head()

,description,isgood
0,사람이 너무 많고 작은 수영장과 조식 수용한계로 모두 포기하고 옆의 아이파크몰에서 ...,0
1,방도크고 깨끗하여 아주 좋았어요,1
2,매년여름휴가철마다 찾는곳이예요 너무 좋아요 점점 더 좋아지는듯 직원분들도 너무 친절...,1
3,여수에서는 제일 유명한 호텔이래요 호텔 stay 가 필요하다면 소노캄 여수도 괜찮은...,1
4,가격대비 훌륭하지만 아무래도 오래된 느낌이 많이 드네요겉이불은 세탁하니까 깨끗히나 ...,1


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53964 entries, 0 to 53963
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  53964 non-null  object
 1   isgood       53964 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 843.3+ KB


In [ ]:
# test_data.info()

# 토큰화

In [ ]:
docs = train_data['description']

In [ ]:
mecab = Mecab()
mecab.morphs(docs[0])

['사람',
 '이',
 '너무',
 '많',
 '고',
 '작',
 '은',
 '수영장',
 '과',
 '조식',
 '수용',
 '한계',
 '로',
 '모두',
 '포기',
 '하',
 '고',
 '옆',
 '의',
 '아이파크몰',
 '에서',
 '그냥',
 '식사',
 '함']

In [ ]:
type(docs)

pandas.core.series.Series

In [ ]:
# 전체 문장을 토큰화 후 tokenized_docs에 저장
tokenized_docs = docs.apply(mecab.morphs)

In [ ]:
# import joblib
# joblib.dump(tokenized_docs, "./model/hotels_tokenized_docs")

In [ ]:
# tokenized_docs = joblib.load("./model/hotels_tokenized_docs")

In [ ]:
tokenized_docs[0]

['사람',
 '이',
 '너무',
 '많',
 '고',
 '작',
 '은',
 '수영장',
 '과',
 '조식',
 '수용',
 '한계',
 '로',
 '모두',
 '포기',
 '하',
 '고',
 '옆',
 '의',
 '아이파크몰',
 '에서',
 '그냥',
 '식사',
 '함']

In [ ]:
# 단어 인덱스 생성
token = Tokenizer(lower=False)
token.fit_on_texts(tokenized_docs)
print(len(token.word_index))

30149


In [ ]:
# 문장 백터화
X = token.texts_to_sequences(tokenized_docs)
print(X[0])

[200, 1, 30, 81, 2, 146, 10, 168, 56, 51, 3466, 3762, 47, 167, 1858, 3, 2, 231, 32, 3068, 27, 205, 221, 71]


In [ ]:
y = train_data['isgood']
y

,isgood
0,0
1,1
2,1
3,1
4,1
...,...
53959,1
53960,1
53961,1
53962,1


In [ ]:
# 가장 긴 문장의 길이 구하기
max_len = max(len(i) for i in X)
print("가장 긴 문장의 길이(패딩에 사용): ", max_len)

가장 긴 문장의 길이(패딩에 사용):  738


In [ ]:
# 패딩
X_padded = pad_sequences(X, maxlen=max_len, padding='post')
print(X_padded[0])

[ 200    1   30   81    2  146   10  168   56   51 3466 3762   47  167
 1858    3    2  231   32 3068   27  205  221   71    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
# 홀드아웃
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_padded, y, test_size=0.3 ,stratify=y, random_state=10)

In [ ]:
# 임베딩에 입력될 단어 수
word_size = len(token.word_index) + 1
print(word_size)

30150


# 양방향 RNN 네트워크를 이용해 텍스트 분석

In [ ]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
birnn = Sequential()
birnn.add(Embedding(word_size, 64, input_length=max_len))
birnn.add(Bidirectional(SimpleRNN(128, return_sequences=True, activation='tanh')))
birnn.add(Dropout(0.5))
birnn.add(SimpleRNN(64, activation='tanh'))
birnn.add(Dropout(0.5))
birnn.add(Dense(32, activation='relu'))
birnn.add(Dense(1, activation='sigmoid'))
birnn.build(input_shape=(None, max_len)) # 입력형태 지정
birnn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 738, 64)             │       1,929,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 738, 256)            │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 738, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 64)                  │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,001,665 (7.64 MB)

 Trainable params: 2,001,665 (7.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
birnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelpath = "./hotels_review_birnn.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=10)

In [ ]:
# birnn_history = birnn.fit(X_train, y_train, epochs=1000, batch_size=64,
#                           validation_data=(X_valid, y_valid),
#                           callbacks=[earlystop, checkpoint])

Epoch 1/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 184s 299ms/step - accuracy: 0.7722 - loss: 0.5527 - val_accuracy: 0.7812 - val_loss: 0.5259
Epoch 2/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 154s 261ms/step - accuracy: 0.7797 - loss: 0.5328 - val_accuracy: 0.7812 - val_loss: 0.5257
Epoch 3/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 202s 261ms/step - accuracy: 0.7765 - loss: 0.5348 - val_accuracy: 0.7812 - val_loss: 0.5260
Epoch 4/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 201s 260ms/step - accuracy: 0.7803 - loss: 0.5293 - val_accuracy: 0.7812 - val_loss: 0.5271
Epoch 5/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 202s 259ms/step - accuracy: 0.7833 - loss: 0.5253 - val_accuracy: 0.7812 - val_loss: 0.5255
Epoch 6/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 202s 259ms/step - accuracy: 0.7837 - loss: 0.5252 - val_accuracy: 0.7812 - val_loss: 0.5265
Epoch 7/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 203s 260ms/step - accuracy: 0.7812 - loss: 0.5276 - val_accuracy: 0.7812 - val_loss: 0.5264
Epoch 8/1000
591/591 ━━━━━━━━━━━━━━━━━━━━ 154s 260ms/step - ac

# LSTM와 CNN조합 모델로 분석

In [ ]:
from tensorflow.keras.layers import Dropout, Activation, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=word_size, output_dim=64))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(Conv1D(128, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(Bidirectional(LSTM(256, return_sequences=True, activation='tanh')))
model.add(Dropout(0.5))
model.add(LSTM(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, max_len))
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 738, 64)             │       1,929,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 738, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 734, 128)            │          41,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 183, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 179, 128)            │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 44, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 44, 512)             │         788,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 44, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         328,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,179,777 (12.13 MB)

 Trainable params: 3,179,777 (12.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelpath = "./hotels_review_RNN_CNN.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=10)

In [ ]:
rnn_cnn_history = model.fit(X_train, y_train, epochs=1000, batch_size=128,
                          validation_data=(X_valid, y_valid),
                          callbacks=[earlystop, checkpoint])

Epoch 1/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 29s 60ms/step - accuracy: 0.7691 - loss: 0.5411 - val_accuracy: 0.7812 - val_loss: 0.5272
Epoch 2/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 34s 54ms/step - accuracy: 0.7793 - loss: 0.5309 - val_accuracy: 0.7812 - val_loss: 0.5280
Epoch 3/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 21s 55ms/step - accuracy: 0.7805 - loss: 0.5296 - val_accuracy: 0.7812 - val_loss: 0.5260
Epoch 4/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.7818 - loss: 0.5262 - val_accuracy: 0.7812 - val_loss: 0.5294
Epoch 5/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 16s 55ms/step - accuracy: 0.7831 - loss: 0.5245 - val_accuracy: 0.7812 - val_loss: 0.5259
Epoch 6/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.7827 - loss: 0.5252 - val_accuracy: 0.7812 - val_loss: 0.5265
Epoch 7/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.7830 - loss: 0.5244 - val_accuracy: 0.7812 - val_loss: 0.5308
Epoch 8/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.7808 -

In [ ]:
# !pip install attention

In [ ]:
# from attention import Attention

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, GlobalAveragePooling1D
from tensorflow.keras.layers import Attention

In [ ]:
# 입력 레이어
inputs = Input(shape=(max_len,))

# 임베딩 레이어
x = Embedding(input_dim=word_size, output_dim=64)(inputs)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)   # 양방향 LSTM 과 attention
attention = Attention()([x,x])
x = LSTM(64, return_sequences=True)(x)   # 단방향 LSTM 과 attention
attention = Attention()([x,x])
x = Dropout(0.5)(attention)
x = GlobalAveragePooling1D()(x)  # 차원축소
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 738)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 738, 64)        │      1,929,600 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 738, 64)        │              0 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 738, 256)       │        197,632 │ dropout_8[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, 738, 64)        │         82,176 │ bidirectional_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_3 (Attention)   │ (None, 738, 64)        │              0 │ lstm_5[0][0],          │
│                           │                        │                │ lstm_5[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 738, 64)        │              0 │ attention_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 64)             │              0 │ dropout_9[0][0]        │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 256)            │         16,640 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │         32,896 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 64)             │          8,256 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 32)             │          2,080 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 1)              │             33 │ dense_8[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,269,313 (8.66 MB)

 Trainable params: 2,269,313 (8.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
modelpath = "./hotels_review_RNN_attention.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=10)

In [ ]:
attention_history = model.fit(X_train, y_train, epochs=1000, batch_size=128,
                          validation_data=(X_valid, y_valid),
                          callbacks=[earlystop, checkpoint])

Epoch 1/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 66s 210ms/step - accuracy: 0.7842 - loss: 0.5084 - val_accuracy: 0.7812 - val_loss: 0.5076
Epoch 2/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 61s 206ms/step - accuracy: 0.8037 - loss: 0.4275 - val_accuracy: 0.8762 - val_loss: 0.3094
Epoch 3/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 82s 208ms/step - accuracy: 0.8905 - loss: 0.2748 - val_accuracy: 0.8840 - val_loss: 0.2908
Epoch 4/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 61s 205ms/step - accuracy: 0.9098 - loss: 0.2347 - val_accuracy: 0.8666 - val_loss: 0.3146
Epoch 5/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 83s 209ms/step - accuracy: 0.9184 - loss: 0.2178 - val_accuracy: 0.8847 - val_loss: 0.2835
Epoch 6/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 82s 207ms/step - accuracy: 0.9291 - loss: 0.1925 - val_accuracy: 0.8757 - val_loss: 0.3498
Epoch 7/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 82s 207ms/step - accuracy: 0.9349 - loss: 0.1752 - val_accuracy: 0.8739 - val_loss: 0.3575
Epoch 8/1000
296/296 ━━━━━━━━━━━━━━━━━━━━ 82s 208ms/step - accuracy: 

# 머신러닝 나이브 베이즈와 비교를 위한 분석

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
def tokenizer(x):
  token = mecab.morphs(x)
  return token

In [ ]:
X2 = train_data['description']
y2 = train_data['isgood']

In [ ]:
tfidf_cv = TfidfVectorizer(tokenizer=tokenizer, max_df=0.8, min_df=4, ngram_range=(1,3))
x_tfidf = tfidf_cv.fit_transform(X2)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X2_train, X2_valid, y2_train, y2_valid = train_test_split(x_tfidf, y2, test_size=0.4, stratify=y2, random_state=10)
X2_valid, X2_test, y2_valid, y2_test = train_test_split(X2_valid, y2_valid, test_size=0.5, stratify=y2_valid, random_state=10)

In [ ]:
mnb = MultinomialNB()
mnb.fit(X2_train, y2_train)
pred = mnb.predict(X2_valid)
print(classification_report(y2_valid, pred))
print("="*40, "test result", "="*40)
test_pred = mnb.predict(X2_test)
print(classification_report(y2_test, test_pred))

              precision    recall  f1-score   support

           0       0.92      0.32      0.47      2362
           1       0.84      0.99      0.91      8431

    accuracy                           0.84     10793
   macro avg       0.88      0.65      0.69     10793
weighted avg       0.86      0.84      0.81     10793

======================================== test result ========================================
              precision    recall  f1-score   support

           0       0.93      0.31      0.46      2362
           1       0.84      0.99      0.91      8431

    accuracy                           0.84     10793
   macro avg       0.88      0.65      0.69     10793
weighted avg       0.86      0.84      0.81     10793



In [ ]:
import joblib

In [ ]:
joblib.dump(tfidf_cv, "./tfidf_cv")
joblib.dump(mnb, "./naive_bayes_hotels_model")

['./naive_bayes_hotels_model']

In [ ]:
joblib.dump(token, "./token_dl")

['./token_dl']

In [ ]:
test_data = pd.read_csv("/content/hotelscom_review_test.csv")

In [ ]:
test_data.head()

,description,isgood
0,쾌적한 시설과 특히 금진온천은 저에게는 특별한추억이었습니다,1
1,관광호텔로 알고 예약 후 직접 방문시 주변이 모두 모텔주변에 위치하였으며 관광호텔급...,0
2,신축이라 그런지 아주 청결합니다 조명은 여러가지 색깔이 들어오는데 다 켜면 잘 조화...,1
3,맨몸으로들어간 호텔에 치솔과 면도기가 없어서 황당했습니다 해외에도 다있고 국내 모텔...,1
4,도어락이 제대로 작동되지 않았고텔레비전도 아주 낡아서 중간에 소리도 안 들리고 화면...,0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35977 entries, 0 to 35976
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  35977 non-null  object
 1   isgood       35977 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 562.3+ KB


In [ ]:
import joblib

In [ ]:
docs2 = test_data['description']
mecab = Mecab()
tokenized_docs2 = docs2.apply(mecab.morphs)

In [ ]:
# 토큰화
token2 = joblib.load("./token_dl")

In [ ]:
# 문장 벡터화
X3 = token2.texts_to_sequences(tokenized_docs2)
X3[0]

[277, 17, 49, 56, 199, 9094, 720, 10, 283, 344, 6, 1208, 17, 857, 1, 16, 7]

In [ ]:
y3 = test_data['isgood']

In [ ]:
# 모델 훈련시 사용한 최대 길이
train_max_len = 738

In [ ]:
# 모델 훈련시 사용한 최대 길이로 패딩작업
X3_padded = pad_sequences(X3, maxlen=train_max_len, padding='post')
print(X3_padded[0])

[ 277   17   49   56  199 9094  720   10  283  344    6 1208   17  857
    1   16    7    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
# 임베딩에 입력될 단어 수
train_word_size = len(token2.word_index) + 1
print(train_word_size)

30150


# 학습된 모델을 통해서 테스트 데이터 분석

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
test_birnn = load_model("/content/hotels_review_birnn.keras")
test_birnn_pred = test_birnn.predict(X3_padded)
test_birnn_pred

1125/1125 ━━━━━━━━━━━━━━━━━━━━ 134s 118ms/step


array([[0.776563],
       [0.776563],
       [0.776563],
       ...,
       [0.776563],
       [0.776563],
       [0.776563]], dtype=float32)

In [ ]:
test_birnn_pred_df = pd.DataFrame(test_birnn_pred)
test_birnn_pred_df[0] = test_birnn_pred_df[0].apply(lambda x: 1 if x > 0.5 else 0)
test_birnn_pred_df

,0
0,1
1,1
2,1
3,1
4,1
...,...
35972,1
35973,1
35974,1
35975,1


In [ ]:
print(classification_report(y3, test_birnn_pred_df))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      7873
           1       0.78      1.00      0.88     28104

    accuracy                           0.78     35977
   macro avg       0.39      0.50      0.44     35977
weighted avg       0.61      0.78      0.69     35977



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# RNN + CNN 모델로 테스트

In [ ]:
test_rnn_cnn = load_model("/content/hotels_review_RNN_CNN.keras")
test_rnn_cnn_pred = test_rnn_cnn.predict(X3_padded)


1125/1125 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step


In [ ]:
test_rnn_cnn_pred_df = pd.DataFrame(test_rnn_cnn_pred)
test_rnn_cnn_pred_df[0] = test_rnn_cnn_pred_df[0].apply(lambda x: 1 if x > 0.5 else 0)
test_rnn_cnn_pred_df
print(classification_report(y3, test_rnn_cnn_pred_df))

              precision    recall  f1-score   support

           0       0.81      0.59      0.68      7873
           1       0.89      0.96      0.93     28104

    accuracy                           0.88     35977
   macro avg       0.85      0.78      0.80     35977
weighted avg       0.88      0.88      0.87     35977



# RNN+Attention 모델로 테스트

In [ ]:
test_rnn_attention = load_model("/content/hotels_review_RNN_attention.keras")
test_rnn_attention_pred = test_rnn_attention.predict(X3_padded)
test_rnn_attention_pred_df = pd.DataFrame(test_rnn_attention_pred)
test_rnn_attention_pred_df[0] = test_rnn_attention_pred_df[0].apply(lambda x: 1 if x > 0.5 else 0)
test_rnn_attention_pred_df
print(classification_report(y3, test_rnn_attention_pred_df))

1125/1125 ━━━━━━━━━━━━━━━━━━━━ 36s 31ms/step
              precision    recall  f1-score   support

           0       0.79      0.62      0.70      7873
           1       0.90      0.95      0.93     28104

    accuracy                           0.88     35977
   macro avg       0.84      0.79      0.81     35977
weighted avg       0.88      0.88      0.88     35977



# 나이브베이즈 모델로 테스트데이터 분석

In [ ]:
docs2

,description
0,쾌적한 시설과 특히 금진온천은 저에게는 특별한추억이었습니다
1,관광호텔로 알고 예약 후 직접 방문시 주변이 모두 모텔주변에 위치하였으며 관광호텔급...
2,신축이라 그런지 아주 청결합니다 조명은 여러가지 색깔이 들어오는데 다 켜면 잘 조화...
3,맨몸으로들어간 호텔에 치솔과 면도기가 없어서 황당했습니다 해외에도 다있고 국내 모텔...
4,도어락이 제대로 작동되지 않았고텔레비전도 아주 낡아서 중간에 소리도 안 들리고 화면...
...,...
35972,신경에 거슬리는 것 없이 편안한 숙소였습니다
35973,좋긴한데 들어왔을 때 담배 냄새 비슷한게 나서 조금 그랬어요
35974,지금까지 여행가면서 묵었던 숙소 중에 제일 좋았던 곳이예요항상 이런식으로 관리 해주...
35975,방에 온도조절이 인돼 너무건조해서 아침에 일어나니 가족들이 모두 코가막힌거외에는 ...


In [ ]:
tfidf_cv2 = joblib.load("/content/tfidf_cv")
X4_tfidf = tfidf_cv2.transform(docs2)

In [ ]:
mnb2 = joblib.load("/content/naive_bayes_hotels_model")
test_mnb2_pred = mnb2.predict(X4_tfidf)
print(classification_report(y3, test_mnb2_pred))

              precision    recall  f1-score   support

           0       0.91      0.26      0.41      7873
           1       0.83      0.99      0.90     28104

    accuracy                           0.83     35977
   macro avg       0.87      0.63      0.66     35977
weighted avg       0.85      0.83      0.79     35977

